# Microsoft Agent Framework - Jupyter Notebook 실행 가이드

###### https://learn.microsoft.com/en-us/agent-framework/tutorials/agents/run-agent?pivots=programming-language-python

터미널에서 "az login"을 통해 Azure 로그인이 완료된 상태여야합니다.


## 1. 필수 라이브러리 설치

먼저 필요한 패키지들을 설치합니다.

In [24]:
# 필요한 패키지 설치 (한 번만 실행하면 됩니다)
# !pip install agent-framework
# !pip install azure-identity
# !pip install python-dotenv

## 2. 필수 라이브러리 임포트

In [1]:
import asyncio
from agent_framework.azure import AzureOpenAIChatClient
from azure.identity import AzureCliCredential
from dotenv import load_dotenv

print("라이브러리 임포트 완료!")

라이브러리 임포트 완료!


## 3. 환경 설정 및 인증

`.env` 파일을 로드하고 Azure 인증을 설정합니다.

In [2]:
# 환경 변수 로드
load_dotenv()
print("환경 변수 로드 완료!")

# Azure CLI 인증 준비
credential = AzureCliCredential()
print("Azure CLI 인증 설정 완료!")

환경 변수 로드 완료!
Azure CLI 인증 설정 완료!


## 4. Azure OpenAI 채팅 클라이언트 및 에이전트 생성

In [3]:
# Azure OpenAI 채팅 클라이언트 생성 및 에이전트 만들기
agent = AzureOpenAIChatClient(credential=credential).create_agent(
    instructions="You are good at telling jokes.",
    name="Joker"
)

print("농담을 잘하는 'Joker' 에이전트가 생성되었습니다!")

농담을 잘하는 'Joker' 에이전트가 생성되었습니다!


## 5. 에이전트 실행 (Jupyter 노트북 버전)

⚠️ **중요**: Jupyter 노트북에서는 `asyncio.run()`을 사용하지 않고 직접 `await`를 사용합니다.

In [28]:
# Jupyter 노트북에서는 이렇게 실행합니다 (asyncio.run() 없이)
result = await agent.run("Tell me a joke about a pirate. 한국말로 해줘")
print("🏴‍☠️ 해적 농담:")
print(result.text)

🏴‍☠️ 해적 농담:
해적이 배에서 계산기를 꺼내 들면서 뭐라고 했을까요?

“이건 내 보물 계산기야, 숫자마다 금은보화가 숨겨져 있거든!” 🏴‍☠️😂


## 6. 스트리밍 기반의 에이전트 실행

다른 주제로도 농담을 요청해 보세요!

In [30]:
async for update in agent.run_stream("Tell me a joke about a pirate."):
    if update.text:
        print(update.text, end="", flush=True)
print()  # New line after streaming is complete

Why did the pirate go did the pirate go to school?

To to school?

To improve his “arrrrrrrrr improve his “arrrrrrrrrr-tr-ticulationiculation”!
”!


## 7. ChatMessage 가지고 에이전트 실행하기

In [33]:
from agent_framework import ChatMessage, TextContent, UriContent, Role

message = ChatMessage(
    role=Role.USER, 
    contents=[
        TextContent(text="전달된 이미지에 대한 조크를 해줘"),
        UriContent(uri="https://ilarge.lisimg.com/image/21420884/740full-nicolette-king.jpg", media_type="image/jpeg")
    ]
)

result = await agent.run(message)
print(result.text)


얼굴은 안 보이지만, 저렇게 빛나는 머리카락을 보니 '헤어지는 얘기'도 빛나게 해주겠네요! 머리카락이 너무 반짝여서 혹시 '헤어' 관련 마법이라도 부린 건 아닐까요? 😄


In [35]:
from agent_framework import ChatMessage, TextContent, DataContent, Role

# Load image from local file
with open("sample_image.jpg", "rb") as f:
    image_bytes = f.read()

message = ChatMessage(
    role=Role.USER, 
    contents=[
        TextContent(text="What do you see in this image?"),
        DataContent(
            data=image_bytes,
            media_type="image/jpeg"
        )
    ]
)

result = await agent.run(message)
print(result.text)

This image shows the side profile of a person with their hair sleekly pulled back. The lighting is dramatic with bright highlights on the hair and neck, creating striking contrasts. The face area is blurred, but the visible parts suggest a portrait photo taken in a studio setting with artistic use of shadows. 

And here's a lighthearted joke to brighten the mood:

Why don't scientists trust atoms?

Because they make up everything!


## 8. 에이전트와 멀티턴 대화하기

https://learn.microsoft.com/en-us/agent-framework/tutorials/agents/multi-turn-conversation?pivots=programming-language-python

In [4]:
thread = agent.get_new_thread()

result1 = await agent.run("Tell me a joke about a pirate.", thread=thread)
print(result1.text)
print("++++++++++++++++")

result2 = await agent.run("Now add some emojis to the joke and tell it in the voice of a pirate's parrot.", thread=thread)
print(result2.text)


How much did the pirate pay for his peg leg and hook? An arm and a leg. Arrr!
++++++++++++++++
Squawk! How much did the pirate pay for his peg leg and hook? Squawk! An arm and a leg — arrr! 🦜🏴‍☠️🦿🦾💰😂 Squawk! Pieces of eight! Squawk!


In [ ]:
# 멀티턴 대화 예제 - 여러 개의 스레드에서 각각 대화를 독립적으로 유지하기

thread1 = agent.get_new_thread()
thread2 = agent.get_new_thread()

result1 = await agent.run("Tell me a joke about a pirate.", thread=thread1)
print(result1.text)
print("++++++++++++++++")
result2 = await agent.run("Tell me a joke about a robot.", thread=thread2)
print(result2.text)
print("++++++++++++++++")
result3 = await agent.run("Now add some emojis to the joke and tell it in the voice of a pirate's parrot.", thread=thread1)
print(result3.text)
print("++++++++++++++++")
result4 = await agent.run("Now add some emojis to the joke and tell it in the voice of a robot.", thread=thread2)
print(result4.text)


Why do pirates never learn the alphabet? Because they always get stuck at "C."
++++++++++++++++
Why did the robot go on a diet? It had too many bytes.

Want another? Why do robots always get invited to parties? They bring the best backup dancers — they never run out of moves!
++++++++++++++++
Squawk! Why do pirates never learn the alphabet? ’Cause they always get stuck at “C”! Squawk! 🦜🏴‍☠️🌊😂
++++++++++++++++
Beep-boop. I will tell joke. Query: Why did the robot go on a diet? Response: It had too many bytes. 🤖🍽️💾 Beep. Laughter protocol: *ha-ha.*


## 9. 에이전트에서 Function calling 사용하기

https://learn.microsoft.com/en-us/agent-framework/tutorials/agents/function-tools?pivots=programming-language-python

In [8]:
# Annotated 주석을 사용하여 함수 및 해당 위치 매개변수에 대한 추가 설명을 에이전트에 제공합니다.
from typing import Annotated
from pydantic import Field

def get_weather(
    location: Annotated[str, Field(description="The location to get the weather for.")],
) -> str:
    """Get the weather for a given location."""
    return f"The weather in {location} is cloudy with a high of 15°C."

In [ ]:
# 또는 데코레이터를 사용하여 함수 이름과 설명을 명시적으로 지정할 수도 있습니다.

from typing import Annotated
from pydantic import Field
from agent_framework import ai_function

@ai_function(name="weather_tool", description="Retrieves weather information for any location")
def get_weather(
    location: Annotated[str, Field(description="The location to get the weather for.")],
) -> str:
    return f"The weather in {location} is cloudy with a high of 15°C."

In [9]:
import asyncio
from agent_framework.azure import AzureOpenAIChatClient
from azure.identity import AzureCliCredential

agent = AzureOpenAIChatClient(credential=AzureCliCredential()).create_agent(
    instructions="You are a helpful assistant",
    tools=get_weather
)

# 사용
result = await agent.run("서울의 날씨가 어때?")
print(result.text)
# 출력: "서울의 날씨는 흐리고 최고 기온은 15°C입니다."

서울은 현재 흐리고 최고기온은 약 15°C예요. 지금은 쌀쌀하게 느껴질 수 있으니 가벼운 자켓이나 겉옷을 챙기세요. 비 예보는 없지만 정확한 시간별/주간 예보가 필요하면 알려드릴게요. 원하나요?


In [ ]:
# 여러 function을 클래스에 묶어서 만들수도 있습니다.

class WeatherTools:
    def __init__(self):
        self.last_location = None

    def get_weather(
        self,
        location: Annotated[str, Field(description="The location to get the weather for.")],
    ) -> str:
        """Get the weather for a given location."""
        return f"The weather in {location} is cloudy with a high of 15°C."

    def get_weather_details(self) -> int:
        """Get the detailed weather for the last requested location."""
        if self.last_location is None:
            return "No location specified yet."
        return f"The detailed weather in {self.last_location} is cloudy with a high of 15°C, low of 7°C, and 60% humidity."

In [13]:
# 클래스의 메소드도 Function Tool로 사용할 수 있습니다.
tools = WeatherTools()
agent = AzureOpenAIChatClient(credential=AzureCliCredential()).create_agent(
    instructions="You are a helpful assistant",
    tools=[tools.get_weather, tools.get_weather_details]
)


# 사용
result = await agent.run("서울의 날씨가 어때?")
print(result.text)
# 출력: "서울의 날씨는 흐리고 최고 기온은 15°C입니다."

현재 서울은 흐리고 오늘 최고기온은 15°C입니다. 시간별이나 주간 예보, 체감온도 등 더 알고 싶으시면 말씀해 주세요.


### Human-in-the-loop 과 함께 Function 도구 사용하기
https://learn.microsoft.com/en-us/agent-framework/tutorials/agents/function-tools-approvals

에이전트가 함수 호출을 승인하기 위해 사용자 입력을 요구하는 경우 이를 휴먼 인 더 루프 패턴이라고 합니다. 사용자 입력이 필요한 에이전트 실행은 최종 답변으로 완료하는 대신 사용자에게 필요한 입력을 나타내는 응답으로 완료됩니다. 그런 다음 에이전트의 호출자는 사용자로부터 필요한 입력을 가져와 새 에이전트 실행의 일부로 에이전트에 다시 전달할 책임이 있습니다.

In [ ]:
## 아직 파이썬에는 구현되지 않은 기능